## 🗂️ Hierarchical Indexing for Structured Retrieval | RAG100X

This notebook implements **Hierarchical Indexing** — a retrieval strategy designed to reflect the **multi-level structure** of complex content (like documents, chapters, sections) and perform **multi-stage retrieval** that mirrors human information search patterns.

Rather than treating all chunks equally, hierarchical indexing helps navigate large knowledge bases **top-down** — from high-level summaries to fine-grained details — improving both **precision and relevance**.

---

### ✅ What You’ll Learn

- How to design **multi-level indexes** (e.g., section-level, paragraph-level)  
- How to use **two-stage retrieval**: first coarse-grained, then fine-grained  
- How to integrate **LangChain retrievers** to query across hierarchical levels  
- When hierarchical indexing is better than flat vector search  
- How to implement parent-child relationships between document chunks  

---

### 🔍 Real-world Analogy

Imagine you’re looking for a recipe in a cookbook:

> 📖 First, you check the **table of contents** for "Desserts"  
> 📄 Then you jump to that chapter and read the **exact recipe for Tiramisu**

✅ That’s hierarchical retrieval — **start broad, drill down smart**.

---

### 🧠 How Hierarchical Indexing Works Under the Hood

| Step                    | What Happens                                                                 |
|-------------------------|------------------------------------------------------------------------------|
| 1. Document Structuring | Split documents into hierarchical units (chapters → sections → paragraphs)  |
| 2. Indexing             | Create separate vector indexes for higher-level and lower-level chunks      |
| 3. Coarse Retrieval     | Retrieve top nodes from the **higher-level index** (e.g., sections)         |
| 4. Focused Retrieval    | Drill down into associated **child chunks** (e.g., paragraphs within a section) |
| 5. Fusion (Optional)    | Combine results from both levels or rerank them for final context selection |

💡 This allows retrieval to mirror **natural information flow** — top-down and context-aware.

---

### 🚀 Why Hierarchical Indexing Matters

- 🧠 **Preserves document structure**: Honors section-topic boundaries  
- 🔎 **Improves precision**: Narrows search to relevant subsections  
- ⚙️ **Scalable**: Works well with long-form or structured content  
- 💬 **Better context windows**: Reduces irrelevant token bloat for LLMs  

---

### 🏗️ Use Cases Where It Shines

- Long documents (e.g., research papers, policy manuals)  
- Websites or blogs with structured layouts (e.g., guides, FAQs)  
- Legal or financial documents with nested hierarchies  
- Any scenario where context granularity improves retrieval quality  

---

### 🔄 Where This Fits in RAG100X

In earlier projects, you’ve built:

1. Flat retrieval systems with vector stores  
2. Query and context enrichment techniques (HyDE, CEW, Sub-query Decomp)  
3. Reranking pipelines for smarter final selection  

Now, in **Day 16**, you take a structural leap:

> 💡 **Not all chunks are equal — organize before you retrieve.**  
> Hierarchical Indexing brings structure to semantic search.

---


## 📦 Installation & Setup

In [ ]:
# Install required packages
!pip install langchain langchain-openai python-dotenv
import asyncio
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains.summarize.chain import load_summarize_chain
from langchain.docstore.document import Document

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document path

In [ ]:
path = "data/Understanding_Climate_Change.pdf"

## Function to Encode both chunk levels and summary

This function implements a **two-level vectorization pipeline** for PDFs — capturing both **high-level summaries** and **fine-grained content**. Why is this useful?

### 🔍 Why Hierarchical Embedding?

When users query your RAG system, sometimes they need **broad context** (e.g., “What’s the chapter about?”), and other times they need **precise details** (e.g., “What’s the definition of entropy?”). A single-level embedding (just chunks) either:
- Misses global context (if chunks are too narrow), or  
- Loses precision (if summaries are too abstract).  

**Hierarchical vectorization solves this** by storing both:
1. **Summarized representations** of each page (or document section)  
2. **Detailed chunks** of the actual text  

This way, your retriever can serve **both birds with one stone** — high-level overviews and pinpoint answers.

---

### ⚙️ How the Function Works

Here’s a step-by-step explanation of what the function `encode_pdf_hierarchical()` does:

1. **📄 Load PDF pages**:
   - If `is_string=False`, it loads from a real PDF file using `PyPDFLoader`.
   - If `is_string=True`, it assumes the input is raw text (used for testing/debugging).

2. **📝 Summarize Each Page**:
   - Uses **GPT-4o-mini** to generate a concise summary of each page.
   - Handles API rate limits using **exponential backoff** and batching to stay compliant.

3. **🧩 Create Detailed Chunks**:
   - Uses `RecursiveCharacterTextSplitter` to break each page into overlapping chunks.
   - Adds metadata like chunk index and page number.

4. **🔗 Embed Both Levels**:
   - Summaries and chunks are embedded **separately** using `OpenAIEmbeddings`.
   - Two **FAISS vector stores** are created in parallel:
     - One for summaries
     - One for chunks

## 🔁 Retry with Exponential Backoff | Rate Limit Resilience for Async Calls

In production RAG pipelines, API rate limits are a common bottleneck — especially when using services like OpenAI. A **rate limit error** occurs when you send too many requests in a short period, and the server tells you to slow down.

To handle this gracefully, we implement a **retry strategy with exponential backoff**.

---

### 💡 What is Exponential Backoff?

Exponential backoff means:  
> *“Wait a little longer after each failed attempt before retrying — and double the wait time each time.”*

This helps reduce the pressure on the server and increases the chance of a successful retry.

Example retry delays:  
1st retry → wait 1s  
2nd retry → wait 2s  
3rd retry → wait 4s  
... and so on.

---

### 🛠️ Function Purpose

The function `retry_with_exponential_backoff(coroutine)` wraps **any async coroutine** (e.g. calling OpenAI, summarization, embedding) and retries it if it hits a `RateLimitError`.

- ✅ It tries multiple times (default 5).
- 🧠 After each failure, it waits longer before retrying (exponentially).
- 🛑 If all retries fail, it raises the last exception.

This makes your system **more reliable** and protects against temporary spikes in API usage.

---

### 🔍 Why Async?

The function is `async` so it can `await` both:
- The coroutine you're running.
- The pause/wait time using `await asyncio.sleep(...)` inside `exponential_backoff()`.

This is crucial in concurrent systems — so you **don’t block** the event loop while waiting.

---

### ✅ Usage Example

Suppose you're summarizing documents using OpenAI's API:
```python
summary = await retry_with_exponential_backoff(summarize_doc(doc))


In [ ]:
from openai import RateLimitError
import random

## Helper Functions 
async def exponential_backoff(attempt):
    """
    Implements exponential backoff with jitter.

    Args:
        attempt: The current retry attempt number (starts from 0).

    This function calculates a dynamic wait time before retrying a failed operation.
    It increases wait time exponentially with each retry and adds jitter to prevent
    thundering herd problems (many clients retrying at the same fixed interval).
    """
    # Step 1: Compute the exponential delay (2^attempt)
    # e.g., for attempt = 0 → 1s, attempt = 1 → 2s, attempt = 2 → 4s, etc.
    base_delay = 2 ** attempt

    # Step 2: Add a small random delay (jitter) to avoid simultaneous retries
    # Jitter helps prevent all clients from hitting the server again at the same time
    jitter = random.uniform(0, 1)

    # Step 3: Total wait time = exponential delay + jitter
    wait_time = base_delay + jitter

    # Step 4: Log the wait time (optional, useful for debugging and visibility)
    print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")

    # Step 5: Pause execution asynchronously for the calculated time
    # This prevents blocking the event loop while waiting
    await asyncio.sleep(wait_time)



async def retry_with_exponential_backoff(coroutine, max_retries=5):
    """
    Retries a coroutine using exponential backoff upon encountering a RateLimitError.
    
    Args:
        coroutine: The coroutine to be executed.
        max_retries: The maximum number of retry attempts.
        
    Returns:
        The result of the coroutine if successful.
        
    Raises:
        The last encountered exception if all retry attempts fail.
    """
    for attempt in range(max_retries):
        try:
            # Attempt to execute the coroutine
            return await coroutine
        except RateLimitError as e:
            # If the last attempt also fails, raise the exception
            if attempt == max_retries - 1:
                raise e

            # Wait for an exponential backoff period before retrying
            await exponential_backoff(attempt)

    # If max retries are reached without success, raise an exception
    raise Exception("Max retries reached")

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# Function: Hierarchical PDF Vectorization for RAG

async def encode_pdf_hierarchical(path, chunk_size=1000, chunk_overlap=200, is_string=False):
    """
    Asynchronously encodes a PDF book into a hierarchical vector store using OpenAI embeddings.
    Produces two FAISS vectorstores:
    - One for high-level summaries
    - One for detailed chunks

    Args:
        path (str): Path to the PDF file or text string (if is_string=True)
        chunk_size (int): Approximate size of each chunk
        chunk_overlap (int): Overlap between chunks for continuity
        is_string (bool): Whether input is plain text or a real PDF

    Returns:
        (summary_vectorstore, detailed_vectorstore): FAISS vectorstores at two levels
    """

    # Load the PDF as a list of documents (each page = one doc)
    if not is_string:
        loader = PyPDFLoader(path)
        documents = await asyncio.to_thread(loader.load)
    else:
        # When working with plain text (e.g., testing), split it manually
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        documents = text_splitter.create_documents([path])

    # Set up the LLM summarizer for page-level summaries
    summary_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
    summary_chain = load_summarize_chain(summary_llm, chain_type="map_reduce")

    async def summarize_doc(doc):
        """
        Summarizes one document (page-level) with retry logic.
        Returns a Document object containing the summary.
        """
        summary_output = await retry_with_exponential_backoff(summary_chain.ainvoke([doc]))
        summary = summary_output['output_text']
        return Document(
            page_content=summary,
            metadata={
                "source": path,
                "page": doc.metadata.get("page", 0),
                "summary": True
            }
        )

    # Process documents in batches to avoid OpenAI rate limits
    batch_size = 5
    summaries = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        batch_summaries = await asyncio.gather(*[summarize_doc(doc) for doc in batch])
        summaries.extend(batch_summaries)
        await asyncio.sleep(1)  # Pause between batches for safety

    # Now, split the full document into smaller detailed chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    detailed_chunks = await asyncio.to_thread(text_splitter.split_documents, documents)

    # Add metadata to each chunk: page number, chunk ID, and label it as non-summary
    for i, chunk in enumerate(detailed_chunks):
        chunk.metadata.update({
            "chunk_id": i,
            "summary": False,
            "page": int(chunk.metadata.get("page", 0))
        })

    # Load the embedding model
    embeddings = OpenAIEmbeddings()

    # Function to create vectorstore from any set of documents
    async def create_vectorstore(docs):
        return await retry_with_exponential_backoff(
            asyncio.to_thread(FAISS.from_documents, docs, embeddings)
        )

    # Create both vector stores in parallel: summaries and chunks
    summary_vectorstore, detailed_vectorstore = await asyncio.gather(
        create_vectorstore(summaries),
        create_vectorstore(detailed_chunks)
    )

    return summary_vectorstore, detailed_vectorstore

## 🧠 Encode the PDF book to both document-level summaries and detailed chunks if the vector stores do not exist

This block encodes a PDF into **two levels of granularity** using a hierarchical strategy:

1. **Document-Level Summaries** — for high-level overview and fast top-down retrieval.
2. **Detailed Chunks** — fine-grained sections used for in-depth retrieval.

The system first checks whether pre-computed vector stores already exist:
- If **yes**, it loads them using FAISS and `OpenAIEmbeddings`, saving time and compute.
- If **no**, it calls an async function `encode_pdf_hierarchical(path)` to:
   - Split and embed the PDF into both summary and detailed levels.
   - Save the resulting `FAISS` vector stores locally for future use.

This structure allows hybrid querying:
- Use summaries for coarse filtering or reranking.
- Use detailed chunks for final generation or evidence extraction.

> 📂 Vector stores are saved to:  
> `../vector_stores/summary_store` and `../vector_stores/detailed_store`


In [ ]:
# Check if both summary and detailed FAISS vector stores already exist locally
if os.path.exists("../vector_stores/summary_store") and os.path.exists("../vector_stores/detailed_store"):
    # Initialize the OpenAI embedding model
    embeddings = OpenAIEmbeddings()

    # Load the summary-level vector store from disk
    summary_store = FAISS.load_local(
        "../vector_stores/summary_store",
        embeddings,
        allow_dangerous_deserialization=True  # Required for secure deserialization
    )

    # Load the detailed-level vector store from disk
    detailed_store = FAISS.load_local(
        "../vector_stores/detailed_store",
        embeddings,
        allow_dangerous_deserialization=True
    )

else:
    # If vector stores do not exist, encode the PDF hierarchically (summary + detailed)
    summary_store, detailed_store = await encode_pdf_hierarchical(path)

    # Save the summary vector store to disk for future reuse
    summary_store.save_local("../vector_stores/summary_store")

    # Save the detailed vector store to disk for future reuse
    detailed_store.save_local("../vector_stores/detailed_store")


## 🔍 Retrieve information according to summary level, and then retrieve information from the chunk level vector store and filter according to the summary level pages

This function implements a **hierarchical retrieval** mechanism using two levels of semantic granularity:

1. **High-level Summary Retrieval**  
   It first queries a vector store of **document-level summaries** to identify relevant regions in the document.
   
2. **Targeted Chunk Retrieval**  
   For each top summary result, it filters and retrieves **fine-grained chunks** from the detailed vector store **within the same page** as the summary.

### Why Hierarchical?
- ✅ Reduces noise by narrowing detailed retrieval to only relevant regions.
- ✅ Improves **precision** in large documents with diverse topics.
- ✅ Mimics how humans search: first skim, then deep dive.

### Parameters:
- `k_summaries`: Number of top summaries to retrieve (coarse filtering).
- `k_chunks`: Number of chunks to retrieve per summary (fine detail).

> 📌 Assumes each summary and chunk includes a `"page"` metadata field for page-level alignment.


In [ ]:
def retrieve_hierarchical(query, summary_vectorstore, detailed_vectorstore, k_summaries=3, k_chunks=5):
    """
    Performs a hierarchical retrieval using the query.

    Args:
        query: The search query string.
        summary_vectorstore: Vector store containing high-level document summaries.
        detailed_vectorstore: Vector store containing detailed text chunks.
        k_summaries: Number of top summaries to retrieve (default is 3).
        k_chunks: Number of detailed chunks to retrieve per summary (default is 5).

    Returns:
        A list of relevant detailed chunks aligned with the most relevant summaries.
    """
    
    # Step 1: Retrieve top-k summaries that match the query
    top_summaries = summary_vectorstore.similarity_search(query, k=k_summaries)
    
    # Initialize a list to collect relevant detailed chunks
    relevant_chunks = []

    # Step 2: For each summary, retrieve detailed chunks from the same page
    for summary in top_summaries:
        # Extract the page number metadata from the summary
        page_number = summary.metadata["page"]

        # Define a filter that matches only chunks from the same page
        page_filter = lambda metadata: metadata["page"] == page_number

        # Search the detailed vector store with the same query,
        # but restrict results to the filtered page
        page_chunks = detailed_vectorstore.similarity_search(
            query,
            k=k_chunks,
            filter=page_filter
        )

        # Add the retrieved chunks to the result list
        relevant_chunks.extend(page_chunks)

    # Return all matched detailed chunks across summaries
    return relevant_chunks


### Demonstrate on a use case

In [ ]:
query = "What is the greenhouse effect?"
results = retrieve_hierarchical(query, summary_store, detailed_store)

# Print results
for chunk in results:
    print(f"Page: {chunk.metadata['page']}")
    print(f"Content: {chunk.page_content}...")  # Print first 100 characters
    print("---")

---

## 📘 Summary & Credits

This notebook is based on the excellent open-source repository [RAG_Techniques by NirDiamant](https://github.com/NirDiamant/RAG_Techniques).  
I referred to that work to understand how the pipeline is structured and then reimplemented the same concept in a **fully self-contained** way, but using recent models — as part of my personal learning journey.

The purpose of this notebook is purely **educational**:  
- To deepen my understanding of Retrieval-Augmented Generation systems  
- To keep a clean, trackable log of what I’ve built and learned  
- And to serve as a future reference for myself or others starting from scratch

To support that, I’ve added clear, concise markdowns throughout the notebook — explaining *why* each package was installed, *why* each line of code exists, and *how* each component fits into the overall RAG pipeline. It’s designed to help anyone (including my future self) grasp the **how** and the **why**, not just the **what**.

## 🔍 Why Use Hierarchical Retrieval in RAG?

In large or structured documents, querying the entire corpus at once often retrieves contextually scattered results.

**Hierarchical Retrieval** addresses this by:
- 🧭 First identifying **relevant regions** (via summaries or TOC-level units)  
- 🔎 Then performing **focused retrieval** within those regions  
- 🧹 Reducing noise from irrelevant sections, even if semantically similar  

---

## 🧠 What’s New in This Version?

This implementation includes:

- 🧱 Two-level vectorstores — one for **summaries**, one for **detailed chunks**  
- 🗂️ Metadata-based filtering to align chunks with their summary’s **page number**  
- 🔁 A hierarchical flow: *summaries ➝ filtered detailed chunks*  
- 🔌 Easily integrable into LangChain or standalone RAG pipelines  

---

## 📈 Inferences & Key Takeaways

- ✅ Ideal for **lengthy PDFs, multi-topic reports, or research papers**  
- 🧠 Mimics how humans search — *skim first, deep dive later*  
- ⚡ Reduces unnecessary token usage and LLM context pollution  
- 🔍 Encourages **targeted grounding**, improving factual consistency in generation  

---

## 🚀 What Could Be Added Next?

- 📊 Evaluate impact on **faithfulness and hallucination rates**  
- 🧪 Extend filtering to **multi-field metadata** (e.g., section, chapter)  
- 🔗 Combine with reranking (e.g., **CrossEncoder**) for even tighter precision  
- 🧠 Add fallback to global retrieval if summaries fail to locate answers  


## 💡 Final Word

This notebook is part of my larger personal project: **RAG100x** — a challenge to build and log my journney in RAG from 0 100 in the coming months.

It’s not built to impress — it’s built to **progress**.  
Everything here is structured to enable **daily iteration**, focused experimentation, and clean documentation.

If you're exploring RAG from first principles, feel free to use this as a scaffold for your own builds. And of course — check out the original repository for broader implementations and ideas.